In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# Using GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. Load and preprocess CIFAR-100
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)

test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

# 2. CNN model
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 32, 32]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # [B, 32, 16, 16]

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # [B, 64, 16, 16]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # [B, 64, 8, 8]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# [B, 128, 8, 8]
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                           # [B, 128, 4, 4]
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),                                # [B, 128*4*4]
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 100)                          # 100 classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Instantiate model
cnn_model = CNNNet().to(device)

# Optimizer and Loss
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# Training loop
def train(model, loader, optimizer, loss_fn, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = loss_fn(logits, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Testing loop
def test(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    print(f"Accuracy: {100 * correct / total:.2f}%")

print("\nTraining CNN Model")
train(cnn_model, train_loader, optimizer_cnn, loss_fn)
test(cnn_model, test_loader)


Using device: cpu

Training CNN Model
Epoch 1, Loss: 1545.6695
Epoch 2, Loss: 1305.6616
Epoch 3, Loss: 1174.4551
Epoch 4, Loss: 1091.7484
Epoch 5, Loss: 1026.6721
Epoch 6, Loss: 974.1391
Epoch 7, Loss: 927.5361
Epoch 8, Loss: 894.7945
Epoch 9, Loss: 858.8683
Epoch 10, Loss: 827.8533
Accuracy: 42.84%
